In [1]:
from openai import OpenAI
import json

BASE_URL = "http://0.0.0.0:8080/v1"
API_KEY = "sk-xxx"
MODEL_ID = "functionary-7b-v1.4"

client = OpenAI(base_url=BASE_URL, api_key=API_KEY)

# Baseline

Test out a question without using tools as a baseline

In [2]:
response = client.chat.completions.create(
    model=MODEL_ID,
    messages=[
        {
            "content": "You are a helpful assistant.",
            "role": "system"
        },
        {
            "content": "What is the capital of France?",
            "role": "user"
        }
    ],
)
for choice in response.choices:
    print(choice.message.content)

The capital of France is Paris.


In [3]:
def chat_completion_request(messages, tools=None, model=MODEL_ID):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            # tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [4]:
messages = [
    {"role": "user", "content": "what's the weather like in Hanoi?"}
]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g., San Francisco, CA",
                    }
                },
                "required": ["location"]
            }
        }
    }
]

In [5]:
chat_response = chat_completion_request(
    messages, tools=tools
)
chat_response

ChatCompletion(id='chatcmpl-5d6011e3-e186-42ab-a998-8f259181c924', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_n1gr0fvnQSMTWJeu6lF56xCS', function=Function(arguments='{"location": "Hanoi"}', name='get_current_weather'), type='function')]))], created=1720034955, model='functionary-7b-v1.4', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=25, prompt_tokens=148, total_tokens=157))

In [6]:
import yaml

print(yaml.dump(chat_response.model_dump(), sort_keys=False, explicit_start=True))


---
id: chatcmpl-5d6011e3-e186-42ab-a998-8f259181c924
choices:
- finish_reason: tool_calls
  index: 0
  logprobs: null
  message:
    content: null
    role: assistant
    function_call: null
    tool_calls:
    - id: call_n1gr0fvnQSMTWJeu6lF56xCS
      function:
        arguments: '{"location": "Hanoi"}'
        name: get_current_weather
      type: function
created: 1720034955
model: functionary-7b-v1.4
object: chat.completion
service_tier: null
system_fingerprint: null
usage:
  completion_tokens: 25
  prompt_tokens: 148
  total_tokens: 157

